In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
from src.utils.Tensorflow_helper import choose_gpu_by_id
from tensorflow.keras.utils import plot_model

# define GPU id to use
GPU_IDS = '0,1'
GPUS = choose_gpu_by_id(GPU_IDS)
print(GPUS)

# jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import helpers
from src.utils.utils_io import Console_and_file_logger, init_config
from src.visualization.Visualize import show_2D_or_3D
from src.data.Dataset import get_img_msk_files_from_split_dir, load_acdc_files, get_train_data_from_df, get_trainings_files
from src.data.Generators import DataGenerator, get_samples
from src.utils.KerasCallbacks import get_callbacks
import src.utils.my_metrics as metr
import cv2


# define experiment name for report, model and log paths + filenames
#EXPERIMENT = '2D/tf2/acdc/combined/2D_NOrot90_dfsplit_6_224_11_fold1'
EXPERIMENT = '3D/wrapper/gcn_and_acdc_exlusive_ax/'
now = datetime.datetime.now()
# image params, change for different input data/architecture
ARCHITECTURE = '3D' # 2D
# path params
DATASET = 'GCN'  # 'acdc' # or 'gcn'
TRAIN_PATH = 'data/raw/{}/{}/train/'.format(DATASET, ARCHITECTURE)
VAL_PATH = 'data/raw/{}/{}/val/'.format(DATASET, ARCHITECTURE)
TEST_PATH = 'data/raw/{}/{}/val/'.format(DATASET, ARCHITECTURE)
DF_DATA_PATH = 'data/raw/gcn_05_2020/SAx_3Ddf_kfold.csv'.format(DATASET, ARCHITECTURE)
#DF_DATA_PATH = 'data/raw/miccai2020/2d_dataset.csv' # miccai special case
DATA_PATH = '/mnt/data/git/cardio/data/raw/gcn_05_2020_sax_excl_ax_patients/2D/'
FOLD = 0

MODEL_PATH = os.path.join('models', EXPERIMENT, str(now.strftime("%Y-%m-%d_%H_%M")))
TENSORBOARD_LOG_DIR = os.path.join('reports/tensorboard_logs', EXPERIMENT,str(now.strftime("%Y-%m-%d_%H_%M")))
CONFIG_PATH = os.path.join('reports/configs/',EXPERIMENT,str(now.strftime("%Y-%m-%d_%H_%M")))
HISTORY_PATH = os.path.join('reports/history/',EXPERIMENT,str(now.strftime("%Y-%m-%d_%H_%M")))

DIM = [80, 224, 224] # network input params
DEPTH = 4 # number of down-/upsampling blocks
FILTERS = 32 # initial number of filters, will be doubled after each downsampling block
#SPACING = [8,1.1, 1,1] # used by sitk, order will be reversed to have the same shape as dim
SPACING = [1.5, 1.5, 1.5] # if resample, resample to this spacing
M_POOL = [2, 2, 2]# used for downsampling and upsampling
F_SIZE = [3, 3, 3] # conv filter size
IMG_CHANNELS = 1
MASK_VALUES = [0, 1, 2, 3]  #channel order: Background, RV, MYO, LV
MASK_CLASSES = len(MASK_VALUES) # no of labels
BORDER_MODE = cv2.cv2.BORDER_CONSTANT
IMG_INTERPOLATION = cv2.INTER_LINEAR
MSK_INTERPOLATION = cv2.INTER_NEAREST
AUGMENT = True
SHUFFLE = True
AUGMENT_GRID = False
RESAMPLE = True

SCALER = 'MinMax' # MinMax Standard or Robust

# training params
#GENERATOR_WORKER = 6# if not set, use batchsize
SEED = 42 # define a seed for the generator shuffle
BATCHSIZE = 32 # 32, 64, 24, 16, 1 for 3D use: 8
INITIAL_EPOCH = 0 # change this to continue training
EPOCHS = 300 # define a maximum numbers of epochs
EPOCHS_BETWEEN_CHECKPOINTS = 5
MONITOR_FUNCTION = 'val_dice_coef_labels'
MONITOR_MODE = 'max'
SAVE_MODEL_FUNCTION = 'val_dice_coef_labels'
SAVE_MODEL_MODE = 'max'
BN_FIRST = False # decide if BN between Conv and activation or afterwards
BATCH_NORMALISATION = True # apply BN or not
USE_UPSAMPLE = True # otherwise use transpose
PAD = 'same' # padding strategy
KERNEL_INIT = 'he_normal' # conv weight initialisation
OPTIMIZER = 'adam' # Adam, Adagrad, RMSprop, Adadelta,  # https://keras.io/optimizers/
ACTIVATION = 'elu' # tf.keras.layers.LeakyReLU(), relu, any non linear activation function
LEARNING_RATE = 0.001 # start with a huge lr to converge fast
DECAY_FACTOR = 0.3 # Define a learning rate decay for the ReduceLROnPlateau callback
MIN_LR = 1e-10 # smaller lr does not improve the model
DROPOUT_min = 0.3 # lower dropout at the shallow layers
DROPOUT_max = 0.5 # higher dropout at the deep layers

metrics = [
    metr.dice_coef_labels,
    metr.dice_coef_myo,
    metr.dice_coef_lv,
    metr.dice_coef_rv
]

LOSS_FUNCTION = metr.bce_dice_loss
#LOSS_FUNCTION = metr.jaccard_distance_loss
#LOSS_FUNCTION = metr.bce_dice_jac_loss

Console_and_file_logger(EXPERIMENT, logging.INFO)

# Define a config for param injection,
# save a serialized version, 
# make sure all paths exist
config = init_config(config=locals(), save=True)
print(config)

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


Using TensorFlow backend.
2020-07-23 20:49:03,664 INFO -------------------- Start --------------------
2020-07-23 20:49:03,665 INFO Working directory: /mnt/data/git/cardio.
2020-07-23 20:49:03,665 INFO Log file: ./logs/3D/wrapper/gcn_and_acdc_exlusive_ax/.log
2020-07-23 20:49:03,665 INFO Log level for console: INFO


['/gpu:0', '/gpu:1']
{'GPU_IDS': '0,1', 'GPUS': ['/gpu:0', '/gpu:1'], 'EXPERIMENT': '3D/wrapper/gcn_and_acdc_exlusive_ax/', 'ARCHITECTURE': '3D', 'DATASET': 'GCN', 'TRAIN_PATH': 'data/raw/GCN/3D/train/', 'VAL_PATH': 'data/raw/GCN/3D/val/', 'TEST_PATH': 'data/raw/GCN/3D/val/', 'DF_DATA_PATH': 'data/raw/gcn_05_2020/SAx_3Ddf_kfold.csv', 'DATA_PATH': '/mnt/data/git/cardio/data/raw/gcn_05_2020_sax_excl_ax_patients/2D/', 'FOLD': 0, 'MODEL_PATH': 'models/3D/wrapper/gcn_and_acdc_exlusive_ax/2020-07-23_20_49', 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/3D/wrapper/gcn_and_acdc_exlusive_ax/2020-07-23_20_49', 'CONFIG_PATH': 'reports/configs/3D/wrapper/gcn_and_acdc_exlusive_ax/2020-07-23_20_49', 'HISTORY_PATH': 'reports/history/3D/wrapper/gcn_and_acdc_exlusive_ax/2020-07-23_20_49', 'DIM': [80, 224, 224], 'DEPTH': 4, 'FILTERS': 32, 'SPACING': [1.5, 1.5, 1.5], 'M_POOL': [2, 2, 2], 'F_SIZE': [3, 3, 3], 'IMG_CHANNELS': 1, 'MASK_VALUES': [0, 1, 2, 3], 'MASK_CLASSES': 4, 'BORDER_MODE': 0, 'IMG_INTE

In [2]:
# Load a previous config and by this a pre-trained model
from ipyfilechooser import FileChooser
config_chooser = FileChooser(os.path.join(os.getcwd(),'reports/configs'), 'config.json')
display(config_chooser)

FileChooser(path='/mnt/data/git/cardio/reports/configs', filename='config.json', show_hidden='False')

In [3]:
# Cell for automation of the model loading process
# load one basemodel for the finetuning task
"""
load past config for model training 
"""
if 'streategy' in locals():
    pass
else:
    # distribute the training with the mirrored data paradigm across multiple gpus if available, if not use gpu 0
    strategy = tf.distribute.MirroredStrategy(devices=config.get('GPUS', ["/gpu:0"]))
if 'config_chooser' in locals():
    config_file  = config_chooser.selected
load = True # change to false, if this pipeline is used without finetuning
# load config with all params into global namespace
from src.models.ModelUtils import load_pretrained_model
if load: # load pretrained model
    with open(config_file, encoding='utf-8') as data_file:
        config_temp = json.loads(data_file.read())
    config_temp['LOSS_FUNCTION'] = config['LOSS_FUNCTION']
    logging.info('Load model from Experiment: {}'.format(config_temp['EXPERIMENT']))
    try:
        with strategy.scope():
            custom_objects = {'loss':LOSS_FUNCTION, 'dice_coef_labels':metr.dice_coef_labels, 'dice_coef_myo':metr.dice_coef_myo, 'dice_coef_lv': metr.dice_coef_lv, 'dice_coef_rv':metr.dice_coef_rv}
            globals()['model'] = load_pretrained_model(config_temp, metrics, comp=False,multigpu=False, custom_objects=custom_objects)
            model.summary()
    except Exception as e:
        logging.error(str(e))
        
else:
    pass
# distribute the training with the mirrored data paradigm across multiple gpus if available, if not use gpu 0
#from src.models.ModelManager_2d3d import get_model, create_comb_shared_unet, create_unet, create_3d_wrapper
#from src.models.ModelManager_2d3d import get_model, create_unet
import src.models.Unets as modelmanager
with strategy.scope():
    # create new model
    logging.info('Create model')
    #model = modelmanager.create_2d_3d_avg_model(config, metrics, supervision=False)
    #model = modelmanager.create_unet(config, metrics, supervision=False)
    #model = modelmanager.create_stacked_unet(config, metrics, supervision=False, unet_2d=model)
    #model = modelmanager.create_stacked_unet_concat_input(config, metrics, supervision=False, unet_2d=model)
    #model = create_comb_shared_unet(config, metrics)
    model = modelmanager.create_3d_wrapper_for_2d_unet(config=config, metrics=metrics, unet_2d=model)
    model.summary()

2020-07-23 20:49:31,120 INFO Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
2020-07-23 20:49:31,122 INFO Load model from Experiment: 2D/ax_sax/gcn_and_acdc_exlusive_ax/
2020-07-23 20:49:31,122 INFO load model with keras api
2020-07-23 20:49:34,046 INFO Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2020-07-23 20:49:34,049 INFO Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2020-07-23 20:49:34,055 INFO Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2020-07-23 20:49:34,059 INFO Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2020-07-23 20:49:34,071 INFO Reduce to /job:localhost/replica:0/task:0/device:

Model: "unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 48) 480         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 48) 192         conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 224, 224, 48) 0           batch_normalization[0][0]        
_______________________________________________________________________________________________

2020-07-23 20:49:35,758 INFO use pre-trained 2d unet


Model: "pretrained-2D-wrapper"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 224, 224 0                                            
__________________________________________________________________________________________________
tf_op_layer_unstack (TensorFlow [(None, 224, 224, 1) 0           input_1[0][0]                    
__________________________________________________________________________________________________
unet (Functional)               (None, 224, 224, 4)  19432324    tf_op_layer_unstack[0][3]        
                                                                 tf_op_layer_unstack[0][65]       
                                                                 tf_op_layer_unstack[0][40]       
                                                                 tf_op_layer_u

In [4]:
json_f = os.path.join(config['MODEL_PATH'],'model.json')
h5_f = os.path.join(config['MODEL_PATH'],'model_weights.h5')

model.save(config['MODEL_PATH'])
model_json = model.to_json()
with open(json_f, "w") as json_file:
    json_file.write(model_json)
#model.save(h5_f)

2020-07-23 20:50:33,446 WARNING From /home/sven/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:110: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
2020-07-23 20:50:33,508 WARNING From /home/sven/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:110: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
2020-07-23 20:51:11,910 INFO Assets written to: models/3D/wrapper/gcn_and_acdc_exlusive_ax/2020-07-23_20_49/assets


38789